# Srock Strategy - Index Enhancement 0002

## 1. Import Library

In [7]:
import dai
import random
import pandas as pd

## 2. Prepare Strategy Data

In [8]:
sd = '2023-12-20'
ed = '2026-02-13'

In [9]:
sql_factor =  f"""
SELECT
    date,
    instrument,
    ps_ttm AS factor
FROM cn_stock_prefactors
WHERE sw2021_level1 IN ('110000', '210000', '220000', '230000', '240000', '270000', '280000', '310000', '320000', '330000', '340000', '350000', '360000', '370000', '410000', '420000', '430000', '440000', '450000', '460000', '470000', '480000', '490000', '510000', '630000', '650000', '710000', '720000', '730000', '740000', '760000', '770000')
AND instrument NOT LIKE '%BJ%'
AND is_hs300 = 1
QUALIFY c_group_pct_rank(sw2021_level2, ps_ttm) < 0.20 
AND c_group_pct_rank(sw2021_level2, total_operating_revenue_yoy_lf) > 0.67
AND net_profit_yoy_lf > 0
"""

In [10]:
sql_trade = f"""
WITH
data_strategy AS (
    {sql_factor}
),
data_filter AS (
    SELECT
        date,
        instrument,
        factor AS score,
        c_rank(factor) AS score_rank,
    FROM data_strategy
    QUALIFY score_rank <= 10
),
data_date AS (
    SELECT
        date,
        instrument,
        score, 
        score_rank, 
        1 / c_sum(1) AS position, 
    FROM data_filter JOIN mldt_cn_stock_calendar_daily USING (date)
    WHERE is_month_end_trade = 1
)
SELECT *
FROM data_date
ORDER BY date, score_rank
"""

In [11]:
df_strategy = dai.query(sql_trade, filters = {"date":[sd, ed]}).df()
df_strategy

,date,instrument,score,score_rank,position
0,2023-12-29,600745.SH,0.870506,1.0,0.20
1,2023-12-29,300122.SZ,2.950247,2.0,0.20
2,2023-12-29,601688.SH,3.551713,3.0,0.20
3,2023-12-29,600958.SH,3.624072,4.0,0.20
4,2023-12-29,002920.SZ,3.725878,5.0,0.20
...,...,...,...,...,...
118,2025-12-31,601878.SH,2.667830,4.0,0.25
119,2026-01-30,600104.SH,0.244023,1.0,0.25
120,2026-01-30,603296.SH,0.539602,2.0,0.25
121,2026-01-30,002920.SZ,2.356172,3.0,0.25


## 3. Backtest Strategy

In [12]:
from bigmodule import M

def BigTrader_Initialize(context):
    from bigtrader.finance.commission import PerOrder
    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))

def BigTrader_Before_Trading(context, data):
    pass

def BigTrader_Handle_Tick(context, tick):
    pass

def BigTrader_Handle_Data(context, data):

    df_now = context.data[context.data["date"] == data.current_dt.strftime("%Y-%m-%d")]

    if len(df_now) == 0:
        return
    
    target_instruments  = list(df_now["instrument"])
    holding_instruments = list(context.get_account_positions().keys())

    for instrument in holding_instruments:
        if instrument not in target_instruments:
            context.order_target_percent(instrument, 0)
        
    for i, x in df_now.iterrows():
        position = 0.0 if pd.isnull(x.position) else float(x.position)
        context.order_target_percent(x.instrument, position)

def BigTrader_Handle_Trade(context, trade):
    pass

def BigTrader_Handle_Order(context, order):
    pass

def BigTrader_After_Trading(context, data):
    pass

BigTrader = M.bigtrader.v34(
    
    data = df_strategy,
    
    start_date = """""",
    end_date   = """""",
    
    initialize           = BigTrader_Initialize,
    before_trading_start = BigTrader_Before_Trading,
    handle_tick          = BigTrader_Handle_Tick,
    handle_data          = BigTrader_Handle_Data,
    handle_trade         = BigTrader_Handle_Trade,
    handle_order         = BigTrader_Handle_Order,
    after_trading        = BigTrader_After_Trading,
    
    capital_base = 1000000  + random.uniform(0, 10),
    frequency="""daily""",
    product_type="""自动""",
    rebalance_period_type="""交易日""",
    rebalance_period_days="""1""",
    rebalance_period_roll_forward=True,
    backtest_engine_mode="""标准模式""",
    before_start_days=0,
    volume_limit=1,
    order_price_field_buy="""open""",
    order_price_field_sell="""open""",
    benchmark="""沪深300指数""",
    
    plot_charts=True,
    debug=False,
    backtest_only=False,
    m_name="""BigTrader"""
) 

[2026-02-18 19:59:16] [info     ] bigtrader.v34 开始运行 ..
[2026-02-18 19:59:16] [info     ] 2023-12-29, 2026-01-30, , , instruments=18
[2026-02-18 19:59:16] [info     ] bigtrader module V2.2.0
[2026-02-18 19:59:16] [info     ] bigtrader engine v0.1.0.post9+g7a244b6 2026-02-10
[2026-02-18 19:59:17] [info     ] backtest done, raw_perf_ds:dai.DataSource("_abc40a853e724f15aca513e2c40a83f0")


[2026-02-18 19:59:17] [info     ] bigtrader.v34 运行完成 [1.230s].
